Notebook to implement population model gradient descent to learn parametesr such as p_sympt, p_severe, p_hosp

In [1]:
import datetime
import turicreate as tc
# import matplotlib.pyplot as plt

state_long = 'California'
state_short = 'CA' 

%load_ext autoreload
%autoreload 2
from PopulationData_ag import PopulationData
from HospitalData_v20210203 import HospitalData
# # import random
import autograd


hd_truthful = HospitalData('merged_data.csv', state_short,'20200711','20210305')
pd_truthful = PopulationData('covidestim.csv', state_long,'20200711', '20210305', forecast=False);

# this list is a list of len 1 since we are not doing mini batches
pd_list_train = [PopulationData(state_short+ '_forecast_after_'+str(20201010) + '.csv',state_long,'20200711',str(20201021), training_mode=True, debug_mode=True)]


Finished parsing file /Users/cuongnguyen/Desktop/cn_autograd_debugging/Archive/merged_data.csv

Parsing completed. Parsed 100 lines in 0.106171 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,int,int,int,int,str,int,str,int,int,int,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/cuongnguyen/Desktop/cn_autograd_debugging/Archive/merged_data.csv

Parsing completed. Parsed 19520 lines in 0.048223 secs.

...

Finished parsing file /Users/cuongnguyen/Desktop/cn_autograd_debugging/Archive/covidestim.csv

Parsing completed. Parsed 100 lines in 0.176881 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/cuongnguyen/Desktop/cn_autograd_debugging/Archive/covidestim.csv

Parsing completed. Parsed 21916 lines in 0.192645 secs.

In [2]:
def get_truthdict_given_mid_end(mid_date,end_date):
#     print(mid_date,end_date, 'mid date end date')
    pd_truthful.start_date = mid_date
    pd_truthful.end_date = end_date
    hd_truthful.start_date = mid_date
    hd_truthful.end_date = end_date
    truthdict = {
    'date':pd_truthful.filtered_data['date'],
    'infections':pd_truthful.filtered_data['infections'],
    'symptomatic':pd_truthful.filtered_data['symptomatic'],
    'severe':pd_truthful.filtered_data['severe'],
    'hosp':hd_truthful.filtered_data['previous_day_admission_adult_covid_confirmed'],
    }

    return truthdict


def add_to_dict(d,g):
    for k in d.keys():
        d[k]=d[k]+g[k]
    return d

def scale_dict_vals(d,s):
    for k in d.keys():
        d[k] = s*d[k]
    return d



import numpy as np
import numpy.random as random
def gradient_descent(pd_list_train, init_params, n_iters=2, step_size=0.001, n_steps_between_print=5, lambda_reg = 1, epsilon_stop=5e-10):
    batch_loss_per_iteration = []
    for n in range(n_iters):
        batch_loss = 0
        for pd in pd_list_train: 
            pd.loss_per_iteration += [0]
        
        pd_batch = random.choice(pd_list_train,replace=False, size=1) # in this case, there is only 1 training duration
        if n == 0:
            new_params = init_params
        else:
            new_params = add_to_dict(new_params, scale_dict_vals(grads, -1*step_size))
        
        
        for i,pd in enumerate(pd_batch): # in this case, there is only 1 training duration
            pd.training_mode=True
            new_grad = pd.get_grad_of_loss(new_params,get_truthdict_given_mid_end(pd.filtered_data['date'][-1], pd.end_date) ,lambda_reg= lambda_reg)
            batch_loss += pd.loss_per_iteration[-1] 
            if i == 0:
                grads = new_grad
            else:
                grads = add_to_dict(grads, new_grad)
        
        grads = scale_dict_vals(grads, 1/len(pd_batch))
        batch_loss_per_iteration += [batch_loss/len(pd_batch)]
        print(batch_loss/len(pd_batch), 'at iteration ', n)

    return new_params, batch_loss_per_iteration

train_params = {'T_serial':5.8,'prob_sympt':0.82,'prob_severe':0.2,'prob_hosp':0.25,'prob_soujourn_inf_alpha':3.41, 'prob_soujourn_inf_beta':0.605,'prob_soujourn_symp_alpha':1.62, 'prob_soujourn_symp_beta':0.218}
new_params = gradient_descent(
    pd_list_train, train_params, n_iters=5, step_size=5e-8, n_steps_between_print=5, lambda_reg=1,epsilon_stop=5e-8)



567.1823638466446 at iteration  0
552.2326386661564 at iteration  1
537.3644899857845 at iteration  2
522.5774274485709 at iteration  3
507.87096378076876 at iteration  4
